### Random Forest Classifier with Pipeline and Hyperparameter Tuning

In [1]:
import seaborn as sns
df = sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [5]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [6]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [7]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time']  = encoder.fit_transform(df['time'])

In [9]:
df.time.unique()

array([0, 1])

In [10]:
## Independent and Dependent Feature
X = df.drop(labels=['time'],axis=1)
y = df.time

In [11]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [12]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X,y, test_size=0.20, random_state=42)

In [14]:
X['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [15]:
#Using Sklearn Pipeline to handle the data cleaning in an automated way

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #handle missing values
from sklearn.preprocessing import StandardScaler #feature scaling
from sklearn.preprocessing import OneHotEncoder # categorical to numerical conversion
from sklearn.compose import ColumnTransformer

In [17]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill', 'tip', 'size']

In [18]:
# Automating Feature Engineering
# Numerical Pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')), # Missing values
        ('scaler', StandardScaler()) #feature scaling
    ]
)

# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')), # Handling Missing Values
        ('onehotencoder', OneHotEncoder()) # Categorical features to numerical 
    ]
)

In [19]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [20]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [21]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

models = {
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()
}

In [27]:
from sklearn.metrics import accuracy_score

In [32]:
def evaluate_model(X_train, y_train, X_test, y_test, models):
    report = {}

    for i in range(len(models)):
        model = list(models.values())[i]
        
        model.fit(X_train,y_train)
        
        y_test_pred = model.predict(X_test)
        
        test_model_score = accuracy_score(y_test, y_test_pred)

        report[list(models.keys())[i]] = test_model_score

    return report

In [33]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [34]:
classifier = RandomForestClassifier()

In [35]:
params = {'max_depth':[3,5,10,None],
          'n_estimators':[100,200,300],
          'criterion':['gini','entropy']
          }

In [36]:
from sklearn.model_selection import RandomizedSearchCV

In [37]:
cv = RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.949 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [38]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'entropy'}

In [42]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
params = {'max_depth':[3,5,10,None],
          'n_estimators':[100,200,300],
          'criterion':['squared_error','absolute_error']
          }

In [45]:
reg = RandomizedSearchCV(regressor,param_distributions=params,verbose=3)

In [46]:
reg.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=absolute_error, max_depth=None, n_estimators=100;, score=0.849 total time=   0.1s
[CV 2/5] END criterion=absolute_error, max_depth=None, n_estimators=100;, score=0.768 total time=   0.0s
[CV 3/5] END criterion=absolute_error, max_depth=None, n_estimators=100;, score=0.953 total time=   0.1s
[CV 4/5] END criterion=absolute_error, max_depth=None, n_estimators=100;, score=0.704 total time=   0.0s
[CV 5/5] END criterion=absolute_error, max_depth=None, n_estimators=100;, score=0.702 total time=   0.1s
[CV 1/5] END criterion=absolute_error, max_depth=3, n_estimators=300;, score=0.811 total time=   0.3s
[CV 2/5] END criterion=absolute_error, max_depth=3, n_estimators=300;, score=0.790 total time=   0.3s
[CV 3/5] END criterion=absolute_error, max_depth=3, n_estimators=300;, score=0.946 total time=   0.3s
[CV 4/5] END criterion=absolute_error, max_depth=3, n_estimators=300;, score=0.602 total time=   0.3s
[CV 5/

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   verbose=3)

In [47]:
reg.best_estimator_

RandomForestRegressor(max_depth=3, n_estimators=200)